In [1]:
import string, json, itertools, spacy
import pandas as pd
from tensorflow import keras
import numpy as np

from spacy.lang.en.stop_words import STOP_WORDS

In [2]:
df = pd.read_json(open("final_dataset/merged_dset.json", "r", encoding="utf8"))
df = df.drop(['url'], axis=1)

In [3]:
# count quantity of entries by theme
theme_entry_count = {}
for entry in df.themes:
    for theme in entry:
        if theme in theme_entry_count.keys():
            theme_entry_count[theme] += 1
        else:
            theme_entry_count[theme] = 1

In [4]:
# remove the least frequent themes if needed
themes_to_remove = []
for key, value in theme_entry_count.items(): 
    if value < 100:
        themes_to_remove.append(key)
        
def remove_themes(themes): # TODO
    result = []
    for theme in themes:
        if theme not in themes_to_remove:
            result.append(theme)
    return result

df.themes = df.themes.apply(remove_themes)

# remove entries without theme
df = df[df['themes'].str.len() != 0]

In [5]:
# get a set of themes
themes = set(itertools.chain.from_iterable(df.themes))

# themes encoding
themes = {list(themes)[i]: i for i in range(len(themes))}

def encode_themes(available_themes):
    result = [0,] * len(themes)
    for i in available_themes:
        result[themes[i]] = 1
    return result

df.themes = df.themes.apply(encode_themes)

In [6]:
# authors encoding
authors = df.author.unique()
authors = {list(authors)[i]: i for i in range(len(authors))}
df.author = [authors[row['author']]for index, row in df.iterrows()]

In [7]:
#  Remove punctuation
table = str.maketrans('', '', string.punctuation)
df['text'] = [row['text'].translate(table) for index, row in df.iterrows()]
df['title'] = [row['title'].translate(table) for index, row in df.iterrows()]
#  Remove stopwords
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in STOP_WORDS))

In [8]:
#  Lemmatization
nlp = spacy.load("en_core_web_sm")
lemmatizer = nlp.get_pipe("lemmatizer")

df['title'] = [
                [token.lemma_ for token in nlp(row['title'])]
                for index, row in df.iterrows()
             ]
df['text'] = [
                [token.lemma_ for token in nlp(row['text'])]
                for index, row in df.iterrows()
             ]

In [9]:
# remove too short texts
texts_len = df['text'].apply(len)
df.drop(df[texts_len<50].index, inplace=True)

In [10]:
max_features=10000

# tokenization
tok = keras.preprocessing.text.Tokenizer(
    num_words=max_features,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True)  
tok.fit_on_texts(df['text'])

In [11]:
# text to number sequences
df['text'] = tok.texts_to_sequences(df['text'])
df['title'] = tok.texts_to_sequences(df['title'])

In [12]:
df

,title,author,year,text,themes
0,"[45, 4, 141, 903]",0,2002,"[2, 2334, 1097, 5, 2334, 298, 9780, 1002, 412,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,[2828],1,2002,"[1, 95, 106, 4385, 3, 144, 291, 33, 1426, 8803...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[209],2,2002,"[106, 20, 719, 21, 1474, 329, 240, 159, 421, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1210, 40, 114, 1989, 18, 2321]",3,2002,"[329, 303, 17, 70, 8, 1, 8, 1, 43, 1345, 1, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[259, 1210]",4,2002,"[9, 649, 76, 140, 544, 206, 679, 179, 107, 44,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
17067,"[151, 18, 2, 5178]",168,1990,"[1, 395, 2499, 331, 545, 8944, 28, 2573, 93, 8...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ..."
17068,[],1172,1990,"[2, 597, 152, 153, 200, 277, 167, 4114, 3080, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
17070,"[15, 114]",3191,1990,"[114, 17, 70, 125, 628, 2, 797, 1, 99, 1133, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17071,"[15, 25, 1, 13]",3434,2005,"[15, 1, 13, 37, 36, 15, 1, 311, 174, 544, 38, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
# export encoded data
data = df.to_json()
with open('../data/data.json', 'w') as f:
    f.write(data)
with open('../data/themes.json', 'w') as f:
    json.dump(themes, f)